In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [59]:
from IPython.display import JSON

In [60]:
from googleapiclient.discovery import build

In [71]:
file = open('API_key.txt')
api_key = file.read()
#api_key

In [72]:
channel_ids = ['UC9HGzFGt7BLmWDqooUbWGBg', #Matthew ID
               'UCYqdbnuvx-jJZByotoNW-5Q', # Apollonia
              ]

In [73]:
api_service_name = "youtube"
api_version = "v3"

# Build API client
youtube = build(api_service_name, api_version, developerKey=api_key)

In [74]:
'''
Function to read through the info of the channels and create a dataframe
METHOD: .channels().list()
'''

def get_channel_stats(youtube, channel_ids):
    data = []
    
    # create request
    request = youtube.channels().list(part="snippet,contentDetails,statistics", id=channel_ids)
    
    response = request.execute()

    # loop through the channel
    for item in response['items']:
        info = {'ChannelName': item['snippet']['title'],
                'Subscribers': item['statistics']['subscriberCount'],
                'TotalViews': item['statistics']['viewCount'],
                'Videos': item['statistics']['videoCount'],
                'PlaylistID': item['contentDetails']['relatedPlaylists']['uploads'],   
               
               }
        data.append(info)
    return(pd.DataFrame(data))



In [75]:
df = get_channel_stats(youtube, channel_ids)
df

,ChannelName,Subscribers,TotalViews,Videos,PlaylistID
0,Apollonia Ponti,682000,105397023,1044,UUYqdbnuvx-jJZByotoNW-5Q
1,Matthew Hussey,3050000,541531749,942,UU9HGzFGt7BLmWDqooUbWGBg


In [76]:
playlistID = 'UU9HGzFGt7BLmWDqooUbWGBg'
request = youtube.playlistItems().list(part="snippet, contentDetails", playlistId=playlistID, maxResults=50) #maximum value by default
response = request.execute()
JSON(response)


<IPython.core.display.JSON object>

In [77]:
'''
Function to create a list of the video IDs of a PlaylistID
METHOD: .playlistItems().list()
'''

def get_videoIDs(youtube, playlistID):
    video_IDs = []   
    
    request = youtube.playlistItems().list(part="snippet, contentDetails", playlistId=playlistID, maxResults=50) #maximum value by default
    response = request.execute()

     # loop through the items
    for item in response['items']:
        video_IDs.append(item['contentDetails']['videoId'])
                        
    next_page_token = response.get('nextPageToken')
    
    while next_page_token is not None:
        request = youtube.playlistItems().list(part="snippet, contentDetails", playlistId=playlistID, maxResults=50, pageToken=next_page_token) #maximum value by default
        response = request.execute()
    
     # loop through the items
        for item in response['items']:
            video_IDs.append(item['contentDetails']['videoId'])
        
        next_page_token = response.get('nextPageToken')

        
    return video_IDs


In [121]:
# retrieve video list for Matthew
playlistID = df.loc[df['ChannelName'] == 'Matthew Hussey', 'PlaylistID'].iloc[0]
VideoList_mat = get_videoIDs(youtube, playlistID) 

# retrieve video list for Apollonia
playlistID = df.loc[df['ChannelName'] == 'Apollonia Ponti', 'PlaylistID'].iloc[0]
VideoList_apo = get_videoIDs(youtube,  playlistID) 
VideoList_mat

['L5l_X4TXrZ8',
 'JgKATVJUY7A',
 'ZQ13ZgSJwic',
 'nHqyyt66dQk',
 'l_Cjd2ya5R0',
 'rMSIPaN25vI',
 'v-7h8fxhTFQ',
 'Xvk_v7uYgPw',
 '10CsOVtc0EE',
 'RpXzWZIZA-s',
 'nDUojvw9SNY',
 'FRZxv1yUghI',
 'x3kP0-AXhio',
 'heCYov5NnqI',
 'KANehpUjFak',
 'EdlO3y2e1q0',
 'aS--Q2UpdPw',
 'uEoyMh9GrZo',
 'oB9yB5-4bXg',
 'MlIwAZHGoVM',
 'XAfcRDSk3ac',
 '1wWUo3K2IQ4',
 'auZ3meqfIM8',
 'pROjRDs2TlU',
 'CqUVDeYSF5s',
 'GyMmQsGgqr4',
 'E3xUfFMCPpk',
 'ChzcXB-P_bA',
 'LPvuuVgqLsc',
 '7XnT4l0msj4',
 'gqhqYvRZawo',
 'hruCvah4G0U',
 'OqJ4CC6fx8w',
 'TPtK8Yu71ds',
 'hwhRa7S5bxs',
 'nc-xSE8MCjU',
 'bBjnAYymUo0',
 'DA48H6tsm4Q',
 'XGnrnh4gjXs',
 'nNiHg4mWYUA',
 '4XTL5GYKrYY',
 'VFLtq0kP_Pk',
 'SZP6Ske_bfM',
 'vm2t1MKrcpQ',
 'qR1-QzGGqPE',
 'lug5X3KKsaY',
 'aHtKFDhSG3A',
 'nMvJnk4uZ4k',
 'Ej3_yYhxbY8',
 'dSe9gzt3xUk',
 'pwJ8taTRU7o',
 'H8yEcflLCkQ',
 'L_qAxl8jhhU',
 'HM8QSJIy96w',
 'aozUJLE0sTk',
 'ZNUTlKyvixU',
 '4bYO9Buh9Eg',
 'Wu01wLV6x2s',
 'Bn9XTBQ8Ufw',
 'Ni977nOzSrs',
 'lyhu0KHsqUo',
 '9HrzWaIYwVk',
 'kB9Lsy

In [120]:
len(VideoList_mat)

943

In [144]:
request = youtube.videos().list(part="snippet,contentDetails,statistics", id = VideoList_mat)
response = request.execute()
JSON(response)

HttpError: <HttpError 400 when requesting https://youtube.googleapis.com/youtube/v3/videos returned "The request specifies an invalid filter parameter.". Details: "[{'message': 'The request specifies an invalid filter parameter.', 'domain': 'youtube.parameter', 'reason': 'invalidFilters', 'location': 'parameters.', 'locationType': 'other'}]">

In [150]:
'''
Function to read through the each video of the channel and create a dataframe
METHOD: .channels().list()
'''

def get_video_details(youtube, VideoList): 
    data = []
    j = 0
    
    # Loop through the video list in batches of 50
    while j < len(VideoList_mat):
        # Create request
        request = youtube.videos().list(part="snippet,contentDetails,statistics", id=','.join(VideoList_mat[j:j+50]))
        
        try:
            response = request.execute()
        except Exception as e:
            print(f"An error occurred: {e}")
            j += 50
            continue
    
        # Loop through the response items
        for item in response.get('items', []):
            info = {
                'VideoID': item['id'],
                'Date/Time': item['snippet']['publishedAt'],
                'Title': item['snippet']['title'],
                'Description': item['snippet']['description'],
                'Tags': item['snippet'].get('tags', []),   # Tags might not always be present
                'Duration': item['contentDetails']['duration'],
                'Definition': item['contentDetails']['definition'],
                'Caption': item['contentDetails']['caption'],
                'Views': item['statistics'].get('viewCount', None),  # Handle missing statistics
                'Likes': item['statistics'].get('likeCount', None),
                'Favorite': item['statistics'].get('favoriteCount', None)
            }
            data.append(info)
        
        j += 50  # Increment by 50 to get the next batch of video IDs
        print(f"Processed up to video index: {j}")
    
    # Convert the collected data into a DataFrame
    return pd.DataFrame(data)
    
    # Print the DataFrame to verify
    display(df)
    print(f"Total videos processed: {len(df)}")

In [151]:
video_mat = get_video_details(youtube, VideoList_mat)
video_mat

Processed up to video index: 50
Processed up to video index: 100
Processed up to video index: 150
Processed up to video index: 200
Processed up to video index: 250
Processed up to video index: 300
Processed up to video index: 350
Processed up to video index: 400
Processed up to video index: 450
Processed up to video index: 500
Processed up to video index: 550
Processed up to video index: 600
Processed up to video index: 650
Processed up to video index: 700
Processed up to video index: 750
Processed up to video index: 800
Processed up to video index: 850
Processed up to video index: 900
Processed up to video index: 950


,VideoID,Date/Time,Title,Description,Tags,Duration,Definition,Caption,Views,Likes,Favorite
0,L5l_X4TXrZ8,2024-05-19T11:45:01Z,"How To AVOID Dating ""Players""","►► Get Vulnerable Stories, Real Insights, and ...","[Matthew Hussey, Matt Hussey, Get The Guy, How...",PT19M53S,hd,false,16608,696,0
1,JgKATVJUY7A,2024-05-15T17:34:12Z,Stop These Insecure Dating Behaviors! - w/ Sab...,"►► Get Vulnerable Stories, Real Insights, and ...","[Matthew Hussey, Matt Hussey, Get The Guy, How...",PT54M46S,hd,false,17225,633,0
2,ZQ13ZgSJwic,2024-05-15T15:00:36Z,"The Problem With Pretending We're ""Fine""",The toll pretending “we’re fine” takes on us c...,"[Matthew Hussey, Matt Hussey, Get The Guy, How...",PT1M,hd,false,18988,1428,0
3,nHqyyt66dQk,2024-05-12T12:05:06Z,THIS Is the Most Dangerous Moment in Dating......,"►► Get Vulnerable Stories, Real Insights and P...","[Matthew Hussey, Matt Hussey, Get The Guy, How...",PT12M50S,hd,false,61248,2493,0
4,l_Cjd2ya5R0,2024-05-11T15:00:42Z,Attracted to the Wrong People?,Why is it that we're drawn to people who treat...,"[Matthew Hussey, Matt Hussey, Get The Guy, How...",PT59S,hd,false,67337,4944,0
...,...,...,...,...,...,...,...,...,...,...,...
938,ndra_xByibg,2010-07-21T09:02:05Z,How To Flirt With A Guy - Surefire Tips For Wo...,►► Confused about what to text him? Just copy ...,"[flirting tips for women, flirting with men, f...",PT2M38S,hd,false,1880217,10228,0
939,zkqYk2rJogs,2010-07-21T08:10:50Z,Powerful Conversational Flirting Tips For Wome...,►► Confused about what to text him? Just copy ...,"[flirting tips for women, flirting, flirting a...",PT2M13S,hd,false,923858,4294,0
940,osLuZa7SvDw,2010-07-16T11:51:38Z,The Best Places To Go On A Date - Avoid Gettin...,►► Confused about what to text him? Just copy ...,"[places to go on a date, dating advice, matthe...",PT1M50S,hd,false,171259,1216,0
941,RJe_C-iF3Bk,2010-07-15T17:43:41Z,How To Tell If A Guy Likes You Instantly - Pro...,►► Confused About Why Men Disappear? My FREE g...,"[how to tell if a guy likes you, how to know i...",PT1M32S,hd,false,2799427,13832,0


In [ ]:
j = 0
data = []
# Loop through the video list in batches of 50
while j < len(VideoList_mat):
    
    
    # create request
    request = youtube.videos().list(part="snippet,contentDetails,statistics", id = VideoList_mat[j:j+50])
    response = request.execute()
    
    # loop through the channel
    for item in response['items']:
        info = { 'VideoID': item['id'],
                'Date/Time': item['snippet']['publishedAt'],
                'Title': item['snippet']['title'],
                'Description': item['snippet']['description'],
                'Tags': item['snippet'].get('tags', []),   # Tags might not always be present
                'Duration': item['contentDetails']['duration'], 
                'Definition': item['contentDetails']['definition'],
                'Caption': item['contentDetails']['caption'], 
                'Views': item['statistics'].get('viewCount', None),  # Handle missing statistics
                'Likes': item['statistics'].get('likeCount', None),
                'Favorite': item['statistics'].get('favoriteCount', None) }
        data.append(info)
    j+=51
    print(j) # Increment by 50 to get the next batch of video IDs


In [147]:
pd.DataFrame(data)

,VideoID,Date/Time,Title,Description,Tags,Duration,Definition,Caption,Views,Likes,Favorite
0,L5l_X4TXrZ8,2024-05-19T11:45:01Z,"How To AVOID Dating ""Players""","►► Get Vulnerable Stories, Real Insights, and ...","[Matthew Hussey, Matt Hussey, Get The Guy, How...",PT19M53S,hd,false,16570,696,0
1,JgKATVJUY7A,2024-05-15T17:34:12Z,Stop These Insecure Dating Behaviors! - w/ Sab...,"►► Get Vulnerable Stories, Real Insights, and ...","[Matthew Hussey, Matt Hussey, Get The Guy, How...",PT54M46S,hd,false,17220,632,0
2,ZQ13ZgSJwic,2024-05-15T15:00:36Z,"The Problem With Pretending We're ""Fine""",The toll pretending “we’re fine” takes on us c...,"[Matthew Hussey, Matt Hussey, Get The Guy, How...",PT1M,hd,false,18984,1428,0
3,nHqyyt66dQk,2024-05-12T12:05:06Z,THIS Is the Most Dangerous Moment in Dating......,"►► Get Vulnerable Stories, Real Insights and P...","[Matthew Hussey, Matt Hussey, Get The Guy, How...",PT12M50S,hd,false,61232,2493,0
4,l_Cjd2ya5R0,2024-05-11T15:00:42Z,Attracted to the Wrong People?,Why is it that we're drawn to people who treat...,"[Matthew Hussey, Matt Hussey, Get The Guy, How...",PT59S,hd,false,67321,4942,0
...,...,...,...,...,...,...,...,...,...,...,...
920,ndra_xByibg,2010-07-21T09:02:05Z,How To Flirt With A Guy - Surefire Tips For Wo...,►► Confused about what to text him? Just copy ...,"[flirting tips for women, flirting with men, f...",PT2M38S,hd,false,1880217,10228,0
921,zkqYk2rJogs,2010-07-21T08:10:50Z,Powerful Conversational Flirting Tips For Wome...,►► Confused about what to text him? Just copy ...,"[flirting tips for women, flirting, flirting a...",PT2M13S,hd,false,923858,4294,0
922,osLuZa7SvDw,2010-07-16T11:51:38Z,The Best Places To Go On A Date - Avoid Gettin...,►► Confused about what to text him? Just copy ...,"[places to go on a date, dating advice, matthe...",PT1M50S,hd,false,171259,1216,0
923,RJe_C-iF3Bk,2010-07-15T17:43:41Z,How To Tell If A Guy Likes You Instantly - Pro...,►► Confused About Why Men Disappear? My FREE g...,"[how to tell if a guy likes you, how to know i...",PT1M32S,hd,false,2799427,13832,0


In [149]:
data = []
j = 0

# Loop through the video list in batches of 50
while j < len(VideoList_mat):
    # Create request
    request = youtube.videos().list(part="snippet,contentDetails,statistics", id=','.join(VideoList_mat[j:j+50]))
    
    try:
        response = request.execute()
    except Exception as e:
        print(f"An error occurred: {e}")
        j += 50
        continue

    # Loop through the response items
    for item in response.get('items', []):
        info = {
            'VideoID': item['id'],
            'Date/Time': item['snippet']['publishedAt'],
            'Title': item['snippet']['title'],
            'Description': item['snippet']['description'],
            'Tags': item['snippet'].get('tags', []),   # Tags might not always be present
            'Duration': item['contentDetails']['duration'],
            'Definition': item['contentDetails']['definition'],
            'Caption': item['contentDetails']['caption'],
            'Views': item['statistics'].get('viewCount', None),  # Handle missing statistics
            'Likes': item['statistics'].get('likeCount', None),
            'Favorite': item['statistics'].get('favoriteCount', None)
        }
        data.append(info)
    
    j += 50  # Increment by 50 to get the next batch of video IDs
    print(f"Processed up to video index: {j}")

# Convert the collected data into a DataFrame
df = pd.DataFrame(data)

# Print the DataFrame to verify
display(df)
print(f"Total videos processed: {len(df)}")

Processed up to video index: 50
Processed up to video index: 100
Processed up to video index: 150
Processed up to video index: 200
Processed up to video index: 250
Processed up to video index: 300
Processed up to video index: 350
Processed up to video index: 400
Processed up to video index: 450
Processed up to video index: 500
Processed up to video index: 550
Processed up to video index: 600
Processed up to video index: 650
Processed up to video index: 700
Processed up to video index: 750
Processed up to video index: 800
Processed up to video index: 850
Processed up to video index: 900
Processed up to video index: 950


,VideoID,Date/Time,Title,Description,Tags,Duration,Definition,Caption,Views,Likes,Favorite
0,L5l_X4TXrZ8,2024-05-19T11:45:01Z,"How To AVOID Dating ""Players""","►► Get Vulnerable Stories, Real Insights, and ...","[Matthew Hussey, Matt Hussey, Get The Guy, How...",PT19M53S,hd,false,16608,696,0
1,JgKATVJUY7A,2024-05-15T17:34:12Z,Stop These Insecure Dating Behaviors! - w/ Sab...,"►► Get Vulnerable Stories, Real Insights, and ...","[Matthew Hussey, Matt Hussey, Get The Guy, How...",PT54M46S,hd,false,17225,633,0
2,ZQ13ZgSJwic,2024-05-15T15:00:36Z,"The Problem With Pretending We're ""Fine""",The toll pretending “we’re fine” takes on us c...,"[Matthew Hussey, Matt Hussey, Get The Guy, How...",PT1M,hd,false,18988,1428,0
3,nHqyyt66dQk,2024-05-12T12:05:06Z,THIS Is the Most Dangerous Moment in Dating......,"►► Get Vulnerable Stories, Real Insights and P...","[Matthew Hussey, Matt Hussey, Get The Guy, How...",PT12M50S,hd,false,61238,2493,0
4,l_Cjd2ya5R0,2024-05-11T15:00:42Z,Attracted to the Wrong People?,Why is it that we're drawn to people who treat...,"[Matthew Hussey, Matt Hussey, Get The Guy, How...",PT59S,hd,false,67337,4944,0
...,...,...,...,...,...,...,...,...,...,...,...
938,ndra_xByibg,2010-07-21T09:02:05Z,How To Flirt With A Guy - Surefire Tips For Wo...,►► Confused about what to text him? Just copy ...,"[flirting tips for women, flirting with men, f...",PT2M38S,hd,false,1880217,10228,0
939,zkqYk2rJogs,2010-07-21T08:10:50Z,Powerful Conversational Flirting Tips For Wome...,►► Confused about what to text him? Just copy ...,"[flirting tips for women, flirting, flirting a...",PT2M13S,hd,false,923858,4294,0
940,osLuZa7SvDw,2010-07-16T11:51:38Z,The Best Places To Go On A Date - Avoid Gettin...,►► Confused about what to text him? Just copy ...,"[places to go on a date, dating advice, matthe...",PT1M50S,hd,false,171259,1216,0
941,RJe_C-iF3Bk,2010-07-15T17:43:41Z,How To Tell If A Guy Likes You Instantly - Pro...,►► Confused About Why Men Disappear? My FREE g...,"[how to tell if a guy likes you, how to know i...",PT1M32S,hd,false,2799427,13832,0


Total videos processed: 943
